# <center> Полезные функции

<center> Сюда буду добавлять полезные функции, которые могут пригодиться в будущем. В комментарии может быть код примера работы или дополнительные фичи.

In [ ]:
from sklearn.cluster import KMeans

# функция, которая принимает количество кластеров для k-means и матрицу с признаками объектов и возвращает инерцию 
def get_inertia(cluster_num, X):
# инициализируем алгоритм кластеризации
    k_means =  KMeans(n_clusters=cluster_num, random_state=42)
# запускаем алгоритм k-means
    k_means.fit(X)
# находим значение инерции
    inertia = k_means.inertia_
# возвращаем значение инерции
    return inertia

# # создаём пустой список для значений инерции
# inertia = []
# # итерируемся по разным размерам кластеров (от 1 до 9) и сохраняем значение инерции для каждого кластера
# for cluster_num in range(1, 10):
# # сохраняем значения
#     inertia.append(get_inertia(cluster_num, X))

# # визуализируем, как менялась инерция в зависимости от количества кластеров
# # задаём названия осям x и y
# plt.xlabel("cluster", fontsize=12)
# plt.ylabel("inertia", fontsize=12)
# # рисуем изменение инерции
# plt.plot([i for i in range(1, 10)], inertia, 'xb-')

In [ ]:
# импортируем метрику силуэта
from sklearn.metrics import silhouette_score

# напишем функцию, как и при подсчете метода локтя
def get_silhouette(cluster_num, X):
    k_means =  KMeans(n_clusters=cluster_num, init='k-means++', n_init=10, random_state=42)
    k_means.fit(X)
# подсчитаем метрику силуэта, передав данные и то, к каким кластерам относятся объекты
    silhouette = silhouette_score(X, k_means.predict(X))
    return silhouette

# # создадим пустой словарь, ключами будут инерция и количество кластеров
# silhouette_res = {"silhouette": [], "cluster": []}

# # выберем нужные данные 
# X = df[['Attack', 'Defense']]

# for cluster_num in range(2, 10):
#     silhouette_res["silhouette"].append(get_silhouette(cluster_num, X))
#     silhouette_res["cluster"].append(cluster_num)
    
# # сохраним в датафрейм значение силуэта и количество кластеров
# silhouette_df = pd.DataFrame(silhouette_res)

# # установим стиль для визуализиции
# sns.set_style("darkgrid")
# # визуализируем зависимость значения инерции от количества кластеров
# sns.lineplot(data=silhouette_df, x="cluster", y="silhouette", marker= "o")

In [ ]:
#Метод межквартильного размаха (метод Тьюки)
#Модифицированная функция с настраиваемыми IQR, и проверкой на логарифмирование
import numpy as np

def outliers_iqr_mod(data, feature, log_scale=False, left=1.5, right=1.5):
    
    """Функция для очистки дата-фрейма от выбросов по методу межквартильного размаха (метода Тьюки).

    Args:
        data (DataFrame): дата-фрейм с данными.
        feature (pd.Series): признак (столбец) из дата-фрейма. 
        log_scale (bool, optional): Логарифмирование в процессе работы функции. Defaults to False.
        left (float, optional): левый интервал, за пределами которого значения считаются выбросами. Defaults to 1.5.
        right (float, optional): правый интервал, за пределами которого значения считаются выбросами. Defaults to 1.5.

    Returns:
        outliners (DataFrame): дата-фрейм со строками, которые функция посчитала выбросами.
        cleaned (DataFtame): очищенный от выбросов дата-фрейм. 
    """    
    
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliners = data[(x < lower_bound)|(x > upper_bound)]
    cleaned = data[(x >= lower_bound)&(x <= upper_bound)]
    return outliners, cleaned

In [ ]:
#метод z-отклонений (метод сигм)
import numpy as np

def outliers_z_score_mod(data, feature, log_scale=False,
                         left=3, right=3):
    
    """Функция для очистки дата-фрейма от выбросов по методу z-отклонений (методу сигм)

    Args:
        data (DataFrame): дата-фрейм с данными.
        feature (pd.Series): признак (столбец) из дата-фрейма. 
        log_scale (bool, optional): Логарифмирование в процессе работы функции. Defaults to False.
        left (float, optional): левый интервал, за пределами которого значения считаются выбросами. Defaults to 3.
        right (float, optional): правый интервал, за пределами которого значения считаются выбросами. Defaults to 3.

    Returns:
        outliners (DataFrame): дата-фрейм со строками, которые функция посчитала выбросами.
        cleaned (DataFtame): очищенный от выбросов дата-фрейм. 
    """    
    
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left*sigma
    upper_bound = mu + right*sigma
    outliers = data[(x<lower_bound) | (x>upper_bound)]
    cleaned = data[(x>lower_bound) & (x<upper_bound)]
    return outliers, cleaned

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def show_corr_heatmap(data, columns_list, title, fontsize=18, method='pearson'):
    
    """Функция для построения тепловой карты корреляций столбцов дата-фрейма.

    Args:
        data (DataFrame): дата-фрейм, по которому нужно построить тепловую карту.
        columns_list (list): список столбцов, по которым нужно построить тепловую карту.
        title (str): будущий заголовок тепловой карты.
        fontsize (int): размер шрифта для заголовка. По умолчанию равен 18.
        method (str, optional): метод корреляции признаков. По умолчанию используется корреляция Пирсона.
    """    
    
    fig_, ax_ = plt.subplots(figsize=(15, 15))
    corr = data[columns_list].corr(method)
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr,
                annot=True,
                linewidths=0.1,
                ax=ax_,
                mask=mask,
                cmap='viridis',
                fmt='.1g')
    ax_.set_title(title, fontsize=fontsize)
    plt.show()